# Docs

## CORS_Lib.ipynb
( CORS_Lib_asof: 2023-0606-1550 )
Gather RINEX data sets from https://geodesy.noaa.gov/cors/rinex/



(c) 2020, 2021. 2022, 2023 C. W. Wright  


**Important Note:** High rate (1 Hz) is avaliable from CORS for only 30 days.  Data older than 30 days, at sampling intervals faster than every 30 seconds, can be had by contacting: https://www.avl.class.noaa.gov/saa/products/search?datatype_family=CORS

** Visit https://geodesy.noaa.gov/CORS/data.shtml for details on sp3 for gps & glonas, and also nav files.

## Todo list
* Add code to trim to desired time range.
* Add code to create a panel widget
* Add code to gather and merge "same day" data. [(see teqc merge post here)](https://postal.unavco.org/pipermail/teqc/2014/001827.html)
* Switch to [Juypter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html)
* Add [multiprocessing](https://pymotw.com/2/multiprocessing/basics.html) to gather all the stations in parallel.

## Change Log & Bug Fixes
* 2023-0608
  * Split up to create a Python package
* 2023-0601
  * Modified install_teqc() to download all versions, and test each one to
    make sure it works.
* 2023-0531
  * Added code to add CORS plots to collection.
* 2023-0530
  * Added mew code to fix the issue with downloading sp3 files.
  * Reorganized directory structure to be abit more like the CORS site.
  * Significant changes to do more from wget.
  * Added code to add the CORS readme.txt and rinex211.txt document.

* 2020-1019
  * Added tool cells for gzip, gunzip, mv, RINEX time span extract
  * Added number of lines selector to the display file cell
  * Consoldated all the defs into one cell to make it easier to use stand-alone
  * Added tool to show the size of a Path_Name

* 2020-1018
  * Fixed problem where teqc wasn't loading and thus not allowing a RINEX file to be cut by time
* 2020-0622
  * Added input validity checking to time input strings.
  * Initial commit.

# Code

## Library Functions

### Imports

In [ ]:
#title imports & Installs: Fetch CORS GNSS data for 1 or more sites on a date. { form-width: "35%", display-mode: "form" }
import datetime
import time
import os
import glob
import inspect
import ipywidgets       as ipw
import multiprocessing
import numpy            as np
import pandas           as pd
import panel            as pn
import re
import requests
import subprocess       as sp

from   bs4              import BeautifulSoup
from   pathlib          import Path

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

### Globals

In [ ]:
CORS_Lib_asof = 'CORS_Lib_asof: 2023-0606-1550'

base_url           = 'https://geodesy.noaa.gov/'
rinex_url          = base_url+'corsdata/rinex/'
coords_url         = base_url+'corsdata/coord/coord_14/'
plots_url          = base_url+'corsdata/Plots/'
station_log_url    = base_url+'corsdata/station_log/'
cors_info_url_list = [ base_url+'/corsdata/readme.txt', 
                      base_url+'/corsdata/RINEX211.txt' ]

teqc_list          = [ 
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64s.zip',
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_CentOSLx86_64d.zip',
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64d.zip',
    'https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64s.zip'
]

test_folder = '/tmp/test/'

if IN_COLAB:
  local_bin = '/usr/local/bin'
else:
  local_bin = f'{str(Path.home())}/bin/'

if __name__ == '__main__':
  print(f'     IN_COLAB: {IN_COLAB}')
  print(f'    local_bin: {local_bin}')
  print(f'CORS_Lib_asof: {CORS_Lib_asof}')
  print(f'     base_url: {base_url}')
  print(f'    rinex_url: {rinex_url}')

wget_return_codes = ('0, No problems occurred.', 
                     '1, Generic error code.',
                     '2, Parse error---for instance, when parsing command-line options, the .wgetrc or .netrc...',
                     '3, File I/O error.',
                     '4, Network failure.',
                     '5, SSL verification failure.',
                     '6, Username/password authentication failure.',
                     '7, Protocol errors.',
                     '8, Server issued an error response.')

     IN_COLAB: False
    local_bin: /home/wright/bin/
CORS_Lib_asof: CORS_Lib_asof: 2023-0606-1550
     base_url: https://geodesy.noaa.gov/
    rinex_url: https://geodesy.noaa.gov/corsdata/rinex/


### def install_teqc():
See: https://www.unavco.org/software/data-processing/teqc/teqc.html

In [ ]:
#title def install_teqc()  { form-width: "35%", display-mode: "form" }
# Get and install teqc
def install_teqc():
  '''
  This function is used to install the teqc software from Unavco. It first
  checks if the teqc software is already installed in the system. If not, 
  it downloads the teqc_Lx86_64s.zip file from Unavco, unzips it, creates
  a local bin directory and moves the teqc file to the local bin directory.
  Finally, it deletes the zip file.

  See: https://www.unavco.org/software/data-processing/teqc/teqc.html

  Parameters:
  None

  Returns:
  A string indicating the state of the installation. 
  It can be either: 'Teqc installed.', 'Teqc was already installed.', or 'teqc install failed.'
  '''
  state = 'initial.'
  os.chdir('/tmp/')
  print(f'install_teqc(): ')
  rv = os.path.isfile(f'{local_bin}teqc')
  if rv == False:
    state = 'teqc install failed.'
    for teqc_zip in teqc_list:
        print(f'Downloading {teqc_zip} and installing Teqc from Unavco.  ')
        sp.run(f'wget {teqc_zip}',                  shell=True)
        teqc_zip_fn = teqc_zip.split( '/')[-1]
        rv = sp.run(f'unzip -o {teqc_zip_fn}',       shell=True)
        sp.run(f'rm -rf {teqc_zip_fn}',              shell=True)
        rv = sp.run(f'/tmp/teqc -help',              shell=True, capture_output=True )
        if rv.returncode == 0 :
            print('Installing teqc.')
            sp.run(f'mkdir -p {local_bin}',          shell=True)
            sp.run(f'mv -f teqc {local_bin}',        shell=True)  
            state = ' Teqc installed.'
            break
        else:
            print(f'Install failed.  rv.return={rv.returncode}')
            print(rv)
  else:
    a = 1;
    state='Teqc was already installed.'
  return state

# Local test and debug code.
if False:
  test_teqc_state = install_teqc()
  print(test_teqc_state)

### def extract_year( date ):

In [ ]:
#=================================================================================
#title def extract_year( date ) { form-width: "35%", display-mode: "form" }
def extract_year( date ):
  Year = date.split('/')[0]
  return Year

### def isTimeFormat(ts):

In [ ]:
#title def isTimeFormat(ts)  { form-width: "35%", display-mode: "form" }
def isTimeFormat(ts):
  """
  isTimeFormat(ts) checks the 'ts' input string for validity. A valid string is in
  this format: '13:34:56'.

  Returns True for a valid string,a nd False for invalid.
  
  """
  t = []
  rv = False;
  try:
      t = time.strptime(ts, '%H:%M:%S')
      rv = True;
  except ValueError:
      rv = False
  return rv

# testing code...
if False:
  for t in ['12:34:56',
            '12:34',
            '01:02:03',
            '23:01:02',
            '25:34:63']:
    print(f'rv:{isTimeFormat(t)}')

### def utctoweekseconds(utc,leapseconds):

In [ ]:
def utctoweekseconds(utc,leapseconds):
  '''
    This function takes two parameters, utc and leapseconds, and returns the
    GPS week, the GPS day, and the seconds and microseconds since the beginning of the GPS week.

    Parameters:
    utc (datetime.datetime): The UTC time to be converted.
    leapseconds (int): The number of leap seconds to be added to the UTC time.

    Returns:
    A tuple containing the GPS week, the GPS day, and the seconds and 
    microseconds since the beginning of the GPS week.

    The GPS week is an integer representing the number of weeks since the 
    beginning of the GPS epoch (1980-01-06 00:00:00).
    The GPS day is an integer representing the number of days since the 
    beginning of the GPS week.
    The seconds and microseconds since the beginning of the GPS week are both
    integers representing the number of seconds and microseconds since the
    beginning of the GPS week.

    Example:
    utc = datetime.datetime(2020, 1, 1, 0, 0, 0)
    leapseconds = 18

    utctoweekseconds(utc, leapseconds)

    Returns: (2086, 0, 0, 0)
  '''
  import datetime, calendar
  datetimeformat = "%Y-%m-%d %H:%M:%S"
  epoch = datetime.datetime.strptime("1980-01-06 00:00:00",datetimeformat)
  tdiff = utc -epoch  + datetime.timedelta(seconds=leapseconds)
  gpsweek = tdiff.days // 7 
  gpsdays = tdiff.days - 7*gpsweek         
  gpsseconds = tdiff.seconds + 86400* (tdiff.days -7*gpsweek) 
  return gpsweek,gpsdays,gpsseconds,tdiff.microseconds


### def compute_day_of_year(date_str):

In [ ]:
#title def compute_day_of_year(date_str) { form-width: "35%", display-mode: "form" }
def compute_day_of_year(date_str):
  '''
    This function computes the day of the year from a given date string in the format 'YYYY/MM/DD'.

    Parameters:
    date_str (str): A string representing a date in the format 'YYYY/MM/DD'.

    Returns:
    A tuple containing the day of the year in the format 'YYYY/DDD', 
    the year as a string, the day as a string, the month as a string, 
    and the day as a string.

    Example:
    compute_day_of_year('2020/04/15')

    Returns:
    ('2020/105', '2020-0415-J105', '2020', '105', '04', '15')

    The returned tuple contains the day of the year in the format 'YYYY/DDD', 
    the year as a string, the day as a string, the month as a string, 
    and the day as a string. The first element of the tuple is the day 
    of the year in the format 'YYYY/DDD', which is the year followed by the 
    day of the year. The second element of the tuple is the year as a string. 
    The third element of the tuple is the day as a string. The fourth element
    of the tuple is the month as a string. The fifth element of the tuple is
    the day as a string.

  '''
  JDay = int(( datetime.datetime.strptime(date_str,'%Y/%m/%d') - datetime.datetime(int(date_str.split('/')[0]),1,1)).days + 1 )
  JDay = f'{JDay:03d}'
  date_parts =  date_str.split('/')
  Year = date_parts[0]
  Month = f'{int(date_parts[1]):02d}'
  Day   = f'{int(date_parts[2]):02d}'
  return f'{Year}/{JDay}', f'{Year}-{Month}{Day}-J{JDay}', Year, JDay, Month, Day
  
if False:
  test_dates = [
      '2023/1/1',
      '2023/01/01',
      '2023/12/31',
      '2023/05/20'
  ]
  print('All returned data:')
  for d in test_dates:
    print( f'{d:12} : { compute_day_of_year(d) } ')

  print('\nCORES path segment only:')
  for d in test_dates:
    print( f'{d:12} : { compute_day_of_year(d)[0]}')


### def getHTMLdocument(url):

In [ ]:
# function to extract html document from given url
def getHTMLdocument(url):
    response = requests.get(url)  # response will be provided in JSON format
    return response.text

if False:
  print('Testing: getHTMLdocument(url):')
  test_html = getHTMLdocument(rinex_url)
  print(test_html)
  print('Done.')

### def download_urls(url_list, root_dir):
Notes:
1. It is important for a url to end in `/` 
when using recursive `-r` and `-np` options to `wget`. See: https://stackoverflow.com/questions/19004809/using-wget-to-recursively-fetch-a-directory-with-no-parent for details.

In [ ]:
def download_urls(url_list, folder, clobber=False, id='', options='', cut_dirs=4):
  '''
download_urls() is a function that downloads a list of URLs to a specified root directory.

Parameters:
url_list (list): A list of URLs to be downloaded.
root_dir (str): The root directory to which the URLs will be downloaded.
clobber (bool): A boolean value indicating whether existing files should be overwritten.
id (str): An optional identifier for the download.
options (str): An optional string of additional wget options.
    
    The function uses the wget command to download the URLs. The wget options used are:
    -nv: Not verbose
    -nc: No clobber (use for OSB files to avoid overwriting 1-Sec with 30-Sec)
    -np: No parent. (Important)
    -nH: Do not create a directory named after the url
    --cut-dirs=4: Remove the first four subdirs
    -R "index.html*": Do not keep the index.html* files.
    -P xxx: xxx will be the root directory

    Wget return codes:
      0   No problems occurred.
      1   Generic error code.
      2   Parse error---for instance, when parsing command-line options, the .wgetrc or .netrc...
      3   File I/O error.
      4   Network failure.
      5   SSL verification failure.
      6   Username/password authentication failure.
      7   Protocol errors.
      8   Server issued an error response.
  '''
  # Wget Options:
  # -nv                Not verbose
  # -nc                No clobber (use for OSB files to avoid 
  #                    overwriting 1-Sec with 30-Sec)
  # -np                No parent. ( Important !!! )
  # -nH                Do not create a directory named after the url
  # --cut-dirs=4       Remove the first four subdirs
  # -R "index.html*"   Do not keep the index.html* files.
  # -P xxx             xxx will be the root directory
  os.makedirs(folder, exist_ok=True)
  log_fn = datetime.datetime.now().strftime(f"{folder}/%Y-%m%d-%H%M%S-CORS-wget-{id}-log.txt")
  url_str = ' '.join(url_list)

  if clobber:
    nc_switch = ''
  else:
    nc_switch = '-nc '

  rv = sp.run( 
      'wget '\
      f'{nc_switch} '\
      f'-o {log_fn} '\
      '-nv '\
      f'-P {folder} '\
      '-np '\
      '-r '\
      '-nH '\
      '--connect-timeout=10 '\
      '--read-timeout=10 '\
      f'--cut-dirs={cut_dirs} '\
      '-R "index.html*,robots.txt,*.tmp" '\
      f'{url_str}',
        shell=True, capture_output=True)
  rv.stderr = log_fn
  rv.id = id
  return rv

if False:
  # Test with some sp3 and nav data.
  test_url      = rinex_url
  fn = f'{test_folder}/sp3_nav'
  test_url_list = CORS_get_Support_url_list(test_url, date='2023/04/28')
  rv = download_urls(test_url_list, folder=fn, id='sp3')
  rv_str = wget_return_codes[rv.returncode]
  print(f'Operation completed. The Wget response was: {rv_str}')

### def CORS_trim_to_time(f, start, stop):

In [ ]:
# Url for data: https://geodesy.noaa.gov/cors/coord/coord_14/
def CORS_trim_to_time(f, start, stop):
  """
Trims a RINEX file (f) to be between the 'start' and 'stop' times given.

This function takes in a file name (f), start time (start) and 
stop time (stop) as parameters and trims the file to the given time range.

The start and stop times are first converted to a format without colons, 
hyphens and periods. The teqc command is then used to trim the file to 
the given time range and the output is stored in a temporary file. 
The temporary file is then moved to the original file name.

Parameters:
f (str): The file name to be trimmed.
start (str): The start time in the format 'YYYY-MM-DD HH:MM:SS.SSS'.
stop (str): The stop time in the format 'YYYY-MM-DD HH:MM:SS.SSS'.

Returns:
rv (CompletedProcess): A CompletedProcess object containing information about the run command.
  """
  st = re.sub('[\:\-\.]', '', start)
  end = re.sub('[\:\-\.]', '', stop)
  print(f'CORS_trim_to_time(): f: {f}   start: {start}  stop: {stop}')
  rv = sp.run(f'teqc +out tmp.txt -st {start} -e {stop} {f}', shell=True)
  sp.run(f'mv tmp.txt {f}', shell=True)
  return rv


### def CORS_get_station_log_url(station_list):

In [ ]:
def CORS_get_station_log_url(station_list):
  '''
    This function takes a list of station names as an argument and 
    returns a list of URLs for the station log files.

    Parameters:
    station_list (list): A list of station names.

    Returns:
    lst (list): A list of URLs for the station log files.
  '''
  lst = []
  for station in station_list:
    station = str.lower(station)
    cords_file = f'{station_log_url}{station}.log.txt'
    lst.append(cords_file)
  return lst

if False:
  station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
  local_test_folder = test_folder
  rv = CORS_get_station_log_url( station_list )
  display(rv)
  print('Done.')


### def down_load_cors_station_log( dir, station_list):

In [ ]:
def down_load_cors_station_log( dir, station_list):
  '''
    This function downloads the log files of the given CORS stations 
    from the NGS website.

    Parameters:
    dir (str): The directory where the log files should be downloaded.
    station_list (list): A list of CORS station names.

    Returns:
    rv (dict): A dictionary containing the station name as the key and 
    the path of the downloaded log file as the value.
  '''
  log_file_list = CORS_get_station_log_url(station_list)
  rv = download_urls(log_file_list, dir, id='station', cut_dirs=1)
  return rv

if False:
  station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
  local_test_folder = test_folder
  rv = down_load_cors_station_log( local_test_folder, station_list )
  print(f'{wget_return_codes[rv.returncode]} See: {rv.stderr}' )
  print('Done.')

### def CORS_get_station_plots_url(station_list):

In [ ]:
def CORS_get_station_plots_url(station_list):
  '''
    This function takes a list of station names as an argument and 
    returns a list of URLs for the corresponding station plot 
    files.

    Parameters:
    station_list (list): A list of station names.

    Returns:
    lst (list): A list of URLs for the corresponding station plot
     files.
  '''
  # Example: https://geodesy.noaa.gov/corsdata/Plots/ab02_14.short.png
  lst = []
  for station in station_list:
    station = str.lower(station)
    short_plots_file = f'{plots_url}{station}_14.short.png'
    long_plots_file  = f'{plots_url}Longterm/{station}_14.long.png'
    lst.append(short_plots_file)
    lst.append(long_plots_file)
  return lst

if False:
  print('Testing: CORS_get_station_plots_url()')
  station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
  rv = CORS_get_station_plots_url( station_list )
  display(rv)
  print('Done.')

### def down_load_cors_plots( dir, station_list):

In [ ]:
def down_load_cors_plots( dir, station_list):
  '''
    This function downloads the plots of the CORS stations from
     the NGS website.

    Parameters:
        dir (str): The directory where the files will be downloaded.
        station_list (list): A list of CORS station names.

    Returns:
        rv (list): A list of the downloaded files.
  '''
  plots_file_list = CORS_get_station_plots_url(station_list)
  rv = download_urls(plots_file_list, dir, id='Plots', cut_dirs=1)
  return rv

if False:
  print('Testing: down_load_cors_plots_coords()')
  station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
  local_test_folder = test_folder
  rv = down_load_cors_plots( local_test_folder, station_list )
  display(rv)
  print(f'{wget_return_codes[rv.returncode]} See: {rv.stderr}' )
  print('Done.')

### def CORS_get_station_coords_url(dir, station_list):

In [ ]:
def CORS_get_station_coords_url(station_list):
  '''
    This function takes a list of station names as an argument and 
    returns a list of URLs for the corresponding station coordinates 
    files.

    Parameters:
    station_list (list): A list of station names.

    Returns:
    lst (list): A list of URLs for the corresponding station coordinates
     files.
  '''
  lst = []
  for station in station_list:
    station = str.lower(station)
    cords_file = f'{coords_url}{station}_14.coord.txt'
    lst.append(cords_file)
  return lst

if False:
  print('Testing: CORS_get_station_coords_url()')
  station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
  rv = CORS_get_station_coords_url( station_list )
  display(rv)
  print('Done.')

### def down_load_cors_station_coords( dir, station_list):

In [ ]:
def down_load_cors_station_coords( dir, station_list):
  '''
    This function downloads the coordinates of the CORS stations from the NGS website.

    Parameters:
        dir (str): The directory where the files will be downloaded.
        station_list (list): A list of CORS station names.

    Returns:
        rv (list): A list of the downloaded files.
  '''
  coords_file_list = CORS_get_station_coords_url(station_list)
  rv = download_urls(coords_file_list, dir, id='coords', cut_dirs=2)
  return rv

if False:
  print('Testing: down_load_cors_station_coords()')
  station_list  = ['ncdu', 'ncbe', 'NCRT', 'loy2']
  local_test_folder = test_folder
  rv = down_load_cors_station_coords( local_test_folder, station_list )
  display(rv)
  print(f'{wget_return_codes[rv.returncode]} See: {rv.stderr}' )
  print('Done.')

### def clean_up_CORS(rdir, date):

In [ ]:
def clean_up_CORS(rdir, date):
  """
  clean_up_CORS(rdir, date)
  Inputs:
  rdir      Directory to start the find command in.
  date      The date of the CORS files to cleanup.

  Outputs:
            Removes unnecessary files.
  Returns:
            None.

  Uses the Linux 'find' command to locate and remove unnecessary files 
  from within a CORS station directory.  
  """
  Year = date.split('/')[0][2:4]
  rv = sp.check_output(f'cd {rdir}; find -name *.md5 -delete; find -name *.{Year}d -delete; find -name *.md5* -delete; find -name *.{Year}S* -delete;', shell=True) 
  return rv


### def CORS_get_Support_url_list( base_url, date, file_types="sp3.gz|n.gz|g.gz" ):
See: https://www.geeksforgeeks.org/beautifulsoup-scraping-link-from-html/

In [ ]:
def CORS_get_Support_url_list( base_url, date, file_types="sp3.gz|n.gz|g.gz" ):
  data_url = base_url+compute_day_of_year( date )[0]
  hdoc = getHTMLdocument(data_url)
  soup = BeautifulSoup( hdoc, 'html.parser')
  urls = []
  for link in soup.find_all('a', attrs={'href' : re.compile(file_types, re.IGNORECASE) } ):
    #print(link.prettify())
    fn = link.get('href')
    url = f'{data_url}/{fn}'
    urls.append(url)
  return urls

# Testing.
if False:
  print('Testing: CORS_get_Support_url_list()')
  print('sp3.gz and n.gz and g.gz')
  #url = 'https://geodesy.noaa.gov/corsdata/rinex/'
  url = rinex_url
  sp3_and_nav_urls = CORS_get_Support_url_list(url, date='2023/05/23')
  for fn in sp3_and_nav_urls:
    print(fn)

  print('\nsp3 only.')
  file_types="sp3.gz"
  sp3_urls = CORS_get_Support_url_list(url, date='2023/05/23', file_types="sp3.gz")
  for fn in sp3_urls:
    print(fn)

  print('\nn.gz and g.gz only.')
  file_types="sp3.gz"
  nav_urls = CORS_get_Support_url_list(url, date='2023/05/23', file_types="n.gz|g.gz")
  for fn in nav_urls:
    print(fn)
  print('Done.')

### def down_load_list_of_obs_files( folder='./', date='' station_list=[] ): 

In [ ]:
def down_load_list_of_obs_files( folder='./', date='', station_list=[] ): 
  '''
    down_load_list_of_obs_files(folder, date, station_list)

    This function downloads a list of observation files from a given URL for a given date and station list.

    Parameters:
    folder (str): The folder where the files should be downloaded.
    date (str): The date for which the files should be downloaded.
    station_list (list): A list of stations for which the files should be downloaded.

    Returns:
    rv (list): A list of the downloaded files.
  '''
  # Create a string from the station list.
  station_url_list = []
  for station in station_list:
    data_url = '/'.join([ rinex_url+compute_day_of_year( date )[0], station.lower()+'/'] )
    station_url_list.append(data_url)
  
  # download the files.
  rv = download_urls(station_url_list, folder=folder,  id='OBS')
  return rv

### def down_load_cors_data( folder = '/', date   = '', station_list = [] ):

In [ ]:
def down_load_cors_data( folder = '/tmp', date = '', station_list = [] ):
  
  def show_progress(msg):
    print(msg)

  show_progress('')
  target_folder = f'{folder}/{compute_day_of_year(date)[1]}-CORS'
  show_progress(f'saving to: {target_folder}')
  rv_lst = []

  show_progress('Downloading info on selected stations.')
  rv = download_urls(cors_info_url_list, target_folder, id='info')
  rv_lst.append(rv)

  show_progress('Downloading selected OBS files.')
  rv = down_load_list_of_obs_files( 
    folder=target_folder, 
    date=date, 
    station_list=station_list
  )
  rv_lst.append(rv)

  show_progress('Downloading sp3 & nav files.')
  url_list = CORS_get_Support_url_list( rinex_url, date = date )
  sp3_nav_folder = '/'.join([target_folder, 'sp3_nav'])
  rv = download_urls(url_list, folder = sp3_nav_folder, id='sp3')
  rv_lst.append(rv)

  show_progress('Downloading station coords, logs, and plots.')
  for func in [down_load_cors_station_coords, 
               down_load_cors_station_log,
               down_load_cors_plots ]:
    rv = func(target_folder, station_list)
    rv_lst.append(rv)

  show_progress('All downloads completed.')
  return rv_lst